In [1]:
### for accessing file from google drive
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/MLlab/oct-mtech/
df =pd.read_csv("diabetes.csv")
df.head()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
# How many missingvalues are mising in each feature
feature_columns = ['Glucose', 'BloodPressure', 'SkinThickness',
    'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age'
]
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing values : {len(df.loc[df[column] == 0])}")

In [ ]:
from sklearn.impute import SimpleImputer
fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)
df[feature_columns] = fill_values.fit_transform(df[feature_columns])

for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing values : {len(df.loc[df[column] == 0])}")

In [ ]:
df.shape

In [ ]:
from sklearn.model_selection import train_test_split
X = df[feature_columns]
y = df.Outcome
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
####pure SVM classifier

from sklearn.svm import SVC
from sklearn.metrics import f1_score
svm = SVC(kernel='linear', random_state=42)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


**SVM used in bagging to create diffetent models**

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import f1_score

svm = SVC(kernel='linear', random_state=42)
bagging = BaggingClassifier(base_estimator=svm, n_estimators=15, random_state=42)
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

**Initialize different classifiers***

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [ ]:
svm = SVC(kernel='linear',random_state=42)
naive_bayes = GaussianNB()
decision_tree = DecisionTreeClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)
base_estimators = [svm, naive_bayes, decision_tree, random_forest]
bagging = BaggingClassifier(base_estimator=None)
bagging.estimators_ = base_estimators
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


**BOOSTING**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.7, random_state=42)
gb.fit(X_train, y_train)
y_pred = gb.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier(n_estimators=100, learning_rate=0.7, random_state=42)
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)

In [ ]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train, y_train)
y_pred = lgb_model.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


**Stacking**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier


('rf', RandomForestClassifier(n_estimators=10, random_state=42)),


In [ ]:
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
              ('knn', KNeighborsClassifier(n_neighbors=5)),
              ('dt', DecisionTreeClassifier(random_state=42))]
meta_estimator = svm
## LogisticRegression()
stacking = StackingClassifier(estimators=estimators, final_estimator=meta_estimator)
stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import numpy as np
# Base classifiers
base_estimators = [
    RandomForestClassifier(n_estimators=10, random_state=42),
    KNeighborsClassifier(n_neighbors=5),
    DecisionTreeClassifier(random_state=42)
]

# Train base classifiers
predictions = []
for estimator in base_estimators:
    estimator.fit(X_train, y_train)
    preds = estimator.predict(X_train)
    predictions.append(preds)

# Convert predictions to features
X_train_stacked = np.column_stack(predictions)

# Neural network as the meta-estimator
meta_model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_stacked.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train meta-estimator
meta_model.fit(X_train_stacked, y_train, epochs=10, batch_size=32)

# Make predictions on test data
predictions_test = []
for estimator in base_estimators:
    preds_test = estimator.predict(X_test)
    predictions_test.append(preds_test)

X_test_stacked = np.column_stack(predictions_test)
y_pred_probs = meta_model.predict(X_test_stacked)
y_pred = (y_pred_probs > 0.5).astype(int)


from sklearn.metrics import f1_score
f1_measure = f1_score(y_test,y_pred)
print("F-measure:", f1_measure)